In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

In [2]:
from modules.lipreading import SentenceReader
from modules.generators import BatchForCTC
from modules.textprocessing import ints2word, sentenceCollapse, word2ints, CODE_BLANK, CODE_SPACE
from glob import glob
import numpy as np
from time import time
from time import strftime as timeformat
from modules.utils import parse_config
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
def error_rate(ypred, ytrue):
    err_mat = (ypred ^ ytrue).astype(np.bool).astype(np.uint8)
    err_sum = err_mat.sum(axis=1)
    #print(err_sum)
    err_mean = err_sum.mean(axis=0)
    return 100*err_mean/ypred.shape[1]

In [4]:
params = parse_config('../config/config-lipnet_300.json')
params['hdf5_data_list'] = [params['hdf5_data_list'][-1]]

In [5]:
sr = SentenceReader(params)
gen = BatchForCTC(params)

In [6]:
samples = 0
ypred = []
ytrue = []
for _ in tqdm(range(params['sample_size']//params['batch_size'])):
    X, Y = None, None
    for inp,_ in gen.next_batch(params['batch_size']):
        X = inp['input']
        Y = inp['label_input']
        break
    
    for i in range(params['batch_size']):
        pred = sr.predict_sentence(X[i])
        og = word2ints(sentenceCollapse(ints2word(Y[i].astype(int))))
        ypred.append(pred)
        ytrue.append(og)

100%|██████████| 4/4 [01:20<00:00, 20.40s/it]


In [7]:
len(ypred)

128

In [8]:
def char_error_rate(ypred, ytrue):
    pred_mat = np.zeros((len(ypred), 32), dtype=int)
    og_mat = np.zeros((len(ytrue), 32), dtype=int)
    for i in range(len(ypred)):
        pred = word2ints(ypred[i])
        og = word2ints(sentenceCollapse(ints2word(ytrue[i])))
        pred_mat[i,:len(pred)] = pred
        og_mat[i,:len(og)] = og
    return error_rate(pred_mat, og_mat)

In [9]:
cer = char_error_rate(ypred, ytrue)
print(cer)

32.6171875


In [10]:
def word_error_rate(ypred, ytrue):
    words = set()
    for pred in ypred:
        words.update(pred.split('_'))
    for og in ytrue:
        words.update(sentenceCollapse(ints2word(og)).split('_'))
    mapping = dict(zip(words, range(len(words))))
    
    pred_mat = np.zeros((len(ypred), 6), dtype=int)
    og_mat = np.zeros((len(ytrue), 6), dtype=int)
    for i in range(len(ypred)):
        pred_words = ypred[i].split('_')
        og_words = sentenceCollapse(ints2word(ytrue[i])).split('_')
        try:
            for j,w in enumerate(pred_words):
                pred_mat[i,j] = mapping[w]
        except IndexError:
            print(i,j,pred_words)
        try:
            for j,w in enumerate(og_words):
                og_mat[i,j] = mapping[w]
        except IndexError:
            print(i,j,og_words)
    return error_rate(pred_mat, og_mat)

In [11]:
wer = word_error_rate(ypred, ytrue)
print(wer)

24 6 ['bin', 'blue', 'at', 'd', 'f', 'svene', 'please']
51.953125


In [12]:
def sentence_error_rate(ypred, ytrue):
    err = 0
    for i in range(len(ypred)):
        pred = ypred[i]
        og = sentenceCollapse(ints2word(ytrue[i]))
        #print(og, pred)
        if pred != og:
            err+=1
    return 100*err/len(ypred)

In [13]:
ser = sentence_error_rate(ypred, ytrue)
print(ser)

100.0
